<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/Proyecto_MW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Simulación por eventos discretos de una clínica M/M/c**

En este cuaderno se modela y simula una **clínica** con varios doctores que atienden pacientes
bajo una dinámica de **línea de espera M/M/c**:

- Llegadas de pacientes: proceso de Poisson con parámetro \\(\lambda\\).
- Tiempos de servicio: distribución exponencial con parámetro \\(\mu\\).
- Número de servidores (doctores): \\(c\\).

La simulación se realiza mediante **eventos discretos** usando una cola de prioridad
(`heapq`) para ordenar los eventos (llegadas y fines de servicio) en el tiempo.

Además de la simulación, se calculan las métricas teóricas del modelo M/M/c usando **Sympy**
y se comparan con los resultados de la simulación:

- \\(W_q\\): tiempo promedio en la cola.
- \\(W\\): tiempo promedio en el sistema.
- \\(L_q\\): número promedio de pacientes en la cola.
- \\(L\\): número promedio de pacientes en el sistema.
- \\(\rho\\): utilización del sistema.


In [ ]:
# Importación de librerías

import heapq
import random
import statistics
import sympy as sp
from dataclasses import dataclass, field
from enum import Enum
from typing import List, Optional

## Clases de datos: pacientes, doctores y eventos

En esta sección se definen las **estructuras básicas** que representan el estado del sistema:

- `EstadoPaciente`: indica si un paciente está en cola, en servicio o ya fue atendido.
- `TipoEvento`: distingue entre eventos de **llegada** y **fin de servicio**.
- `Paciente`: almacena los tiempos clave del paciente (llegada, inicio de servicio, salida).
- `Doctor`: representa a cada servidor (doctor), incluyendo si está ocupado y a quién atiende.
- `Evento`: encapsula un evento en el tiempo, con su tipo y la referencia al paciente/doctor
  involucrado. Se usa como elemento de la cola de prioridad para avanzar la simulación.

In [ ]:
class EstadoPaciente(Enum):
    EN_COLA = 1
    EN_SERVICIO = 2
    ATENDIDO = 3

class TipoEvento(Enum):
    LLEGADA = 1
    FIN_SERVICIO = 2

@dataclass
class Paciente:
    id: int   # identificador único del paciente
    tiempo_llegada: float   # instante en el que llega al sistema
    tiempo_inicio_servicio: float = 0.0   # instante en el que pasa con un doctor
    tiempo_salida: float = 0.0  # instante en el que termina el servicio y abandona el sistema

@dataclass
class Doctor:
    id: int   # identificador del doctor
    ocupado: bool = False   # indica si está atendiendo a un paciente
    paciente_actual: Optional[Paciente] = None  # referencia al paciente que está siendo atendido (si aplica)

@dataclass(order=True)
class Evento:
    tiempo: float   # instante en el que ocurre el evento
    tipo: TipoEvento = field(compare=False)   # tipo de evento (llegada o fin de servicio)
    paciente: Optional[Paciente] = field(compare=False, default=None)   # paciente involucrado en el evento (si aplica)
    doctor: Optional[Doctor] = field(compare=False, default=None)   # doctor involucrado en el evento (si aplica)

## Motor de simulación: clase `ClinicaMMc`

La clase `ClinicaMMc` implementa el **modelo de simulación por eventos discretos** para una clínica M/M/c. Sus responsabilidades principales son:

- Generar las **llegadas** de pacientes usando tiempos entre llegadas exponenciales.
- Administrar la **cola de eventos** (`cola_eventos`) usando `heapq`.
- Asignar pacientes a doctores libres y programar los **fines de servicio**.
- Mantener las estructuras:
  - `cola_pacientes`: pacientes en espera de ser atendidos.
  - `doctores`: lista de servidores (doctores) con su estado.
- Registrar los tiempos de cada paciente para calcular:
  - Tiempo en cola (Wq).
  - Tiempo total en el sistema (W).
- A partir de estos registros, calcular las métricas promedio de desempeño del sistema.

In [ ]:
class ClinicaMMc:
    def __init__(self, lambd, mu, c, tiempo_max):
        self.lambd = lambd
        self.mu = mu
        self.c = c
        self.tiempo_max = tiempo_max

        # Estado del sistema
        self.reloj = 0.0
        self.cola_eventos = []
        self.doctores = [Doctor(i) for i in range(c)]
        self.cola_pacientes = []

        # Métricas (Arrays para guardar la historia de cada pobre alma)
        self.tiempos_espera_cola = []   # Wq
        self.tiempos_sistema = []       # W

    def agendar_evento(self, evento: Evento):
        heapq.heappush(self.cola_eventos, evento)

    def correr(self):
        print(f"Simulación (T_max={self.tiempo_max})")
        # Semilla para reproducibilidad (opcional, quítalo si quieres caos real)
        random.seed(42)

        # Bootstrapping: Primer evento
        self.agendar_evento(Evento(0.0, TipoEvento.LLEGADA))

        while self.cola_eventos and self.reloj < self.tiempo_max:
            evento = heapq.heappop(self.cola_eventos)
            self.reloj = evento.tiempo

            if evento.tipo == TipoEvento.LLEGADA:
                self.manejar_llegada()
            elif evento.tipo == TipoEvento.FIN_SERVICIO:
                self.manejar_fin_servicio(evento)

    def manejar_llegada(self):
        # 1. Crear al paciente
        nuevo_paciente = Paciente(id=len(self.tiempos_sistema) + len(self.cola_pacientes), tiempo_llegada=self.reloj)

        # 2. Programar la llegada del siguiente (Poisson)
        # Si lambda es muy alto, esto explota la RAM en simulaciones infinitas.
        prox_tiempo = self.reloj + random.expovariate(self.lambd)
        if prox_tiempo < self.tiempo_max:
            self.agendar_evento(Evento(prox_tiempo, TipoEvento.LLEGADA))

        # 3. Lógica de asignación
        doctor = self.buscar_doctor_libre()
        if doctor:
            self.iniciar_servicio(nuevo_paciente, doctor)
        else:
            self.cola_pacientes.append(nuevo_paciente)

    def manejar_fin_servicio(self, evento):
        doctor = evento.doctor
        paciente = evento.paciente

        # 1. Estadísticas finales del paciente
        paciente.tiempo_salida = self.reloj

        wq = paciente.tiempo_inicio_servicio - paciente.tiempo_llegada
        w = paciente.tiempo_salida - paciente.tiempo_llegada

        self.tiempos_espera_cola.append(wq)
        self.tiempos_sistema.append(w)

        # 2. Liberar o reasignar doctor
        doctor.ocupado = False
        doctor.paciente_actual = None

        if self.cola_pacientes:
            siguiente = self.cola_pacientes.pop(0)
            self.iniciar_servicio(siguiente, doctor)

    def iniciar_servicio(self, paciente, doctor):
        doctor.ocupado = True
        doctor.paciente_actual = paciente
        paciente.tiempo_inicio_servicio = self.reloj

        # Tiempo de servicio exponencial
        duracion = random.expovariate(self.mu)
        self.agendar_evento(Evento(self.reloj + duracion, TipoEvento.FIN_SERVICIO, paciente, doctor))

    def buscar_doctor_libre(self):
        # Retorna el primer doctor ocioso que encuentre
        for doc in self.doctores:
            if not doc.ocupado:
                return doc
        return None

Sympy:

In [ ]:
def calcular_teorico_sympy(l_val, mu_val, c_val):

    # Simbolos
    lam, mu, c = sp.symbols('lambda mu c', real=True, positive=True)

    # Variables auxiliares
    r = lam / mu
    rho = r / c

    # Validación de estabilidad básica
    utilizacion_real = l_val / (c_val * mu_val)
    if utilizacion_real >= 1:
        print(f"¡ADVERTENCIA! Rho = {utilizacion_real:.2f}. El sistema es inestable.")

    # 1. Probabilidad P0 (Sistema Vacío)
    # Sympy no maneja sumatorias de rango variable en floats fácilmente, lo construimos:
    sumatoria = sum([( (l_val/mu_val)**n ) / sp.factorial(n) for n in range(c_val)])
    ultimo_termino = ( (l_val/mu_val)**c_val ) / (sp.factorial(c_val) * (1 - (l_val/(c_val*mu_val))))

    p0_val = 1 / (sumatoria + ultimo_termino)

    # 2. Lq (Longitud media de cola)
    # Lq = [ (r^c * rho) / (c! * (1-rho)^2) ] * P0
    num = ( (l_val/mu_val)**c_val ) * (l_val/(c_val*mu_val)) * p0_val
    den = sp.factorial(c_val) * (1 - (l_val/(c_val*mu_val)))**2
    lq_val = num / den

    # 3. Resto de métricas (Little)
    wq_val = lq_val / l_val
    w_val = wq_val + (1/mu_val)
    l_val_sys = l_val * w_val

    return {
        "Wq": float(wq_val),
        "W": float(w_val),
        "Lq": float(lq_val),
        "L": float(l_val_sys),
        "Rho": utilizacion_real
    }

if __name__ == "__main__":
    # --- CONFIGURACIÓN ---
    LAMBDA = 4.0     # Pacientes/hora
    MU = 2.0         # Pacientes/hora que atiende un doctor
    C = 3            # Cantidad de doctores
    TIEMPO_SIM = 5000 # Horas simuladas (mientras más alto, más converge al teórico)

    # 1. Ejecutamos la Simulación
    sim = ClinicaMMc(LAMBDA, MU, C, TIEMPO_SIM)
    sim.correr()

    # Métricas de la simulación
    if len(sim.tiempos_sistema) > 0:
        sim_wq = statistics.mean(sim.tiempos_espera_cola)
        sim_w = statistics.mean(sim.tiempos_sistema)
        # Usamos Ley de Little inversa para estimar L y Lq observados
        sim_l = LAMBDA * sim_w
        sim_lq = LAMBDA * sim_wq
    else:
        print("Nadie llegó a la clínica. ¿Es día festivo?")
        exit()

    # 2. Ejecutar Teórico
    teorico = calcular_teorico_sympy(LAMBDA, MU, C)

    # 3. Tabla Comparativa
    print("\n" + "="*60)
    print(f"{'MÉTRICA':<10} | {'SIMULACIÓN':<15} | {'TEÓRICO':<15} | {'ERROR %':<10}")
    print("="*60)

    metricas = [
        ("Wq (Cola)", sim_wq, teorico["Wq"]),
        ("W (Total)", sim_w, teorico["W"]),
        ("Lq (Cola)", sim_lq, teorico["Lq"]),
        ("L (Total)", sim_l, teorico["L"])
    ]

    for nombre, val_sim, val_teo in metricas:
        error = abs(val_sim - val_teo) / val_teo * 100
        print(f"{nombre:<10} | {val_sim:<15.4f} | {val_teo:<15.4f} | {error:<10.2f}")

    print("="*60)
    print(f"Total Pacientes Atendidos: {len(sim.tiempos_sistema)}")
    print(f"Utilización del Sistema (Rho): {teorico['Rho']:.2%}")

Simulación (T_max=5000)

MÉTRICA    | SIMULACIÓN      | TEÓRICO         | ERROR %   
Wq (Cola)  | 0.2197          | 0.2222          | 1.15      
W (Total)  | 0.7188          | 0.7222          | 0.47      
Lq (Cola)  | 0.8787          | 0.8889          | 1.15      
L (Total)  | 2.8753          | 2.8889          | 0.47      
Total Pacientes Atendidos: 19981
Utilización del Sistema (Rho): 66.67%
